## Взаимодействие модальностей

In [ ]:
!pip install datasets jaxtyping better_abc pillow wandb circuitsvis

#### Связи проекта

In [ ]:
import sys
from pathlib import Path

In [ ]:
path = "."
# for kaggle
# path = "/kaggle/input/logitlens4-0/LogitLens/research/logits-dynamics/notebook"

ROOT = Path(path).absolute().parent.parent.parent

DATA = ROOT / "data"

In [ ]:
if ROOT not in sys.path:
    sys.path.append(str(ROOT))

#### Импорт библиотек

In [ ]:
import requests
import torch
from PIL import Image
import circuitsvis as cv
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
from math import ceil
import json

import transformer_lens.utils as utils
from transformer_lens import HookedTransformer

from research.utils import get_data_images, plot_data, plot_data_attention, plot_images

#### Логиты

Создаем модель

In [ ]:
import logging

In [ ]:
torch.set_grad_enabled(False)

device = utils.get_device()

In [ ]:
model_name = "llava-hf/llava-1.5-7b-hf"
last_layer = 32

In [ ]:
model = HookedTransformer.from_pretrained(model_name, device=device)

Загружаем изображения из датасета COCO

In [ ]:
COCO = DATA / "coco_ann2017"
IMAGE_URL_FILE = COCO / "images_urls.json"

In [ ]:
with open(IMAGE_URL_FILE, mode="r") as file:
    dict_images = json.load(file)

images = [Image.open(requests.get(url, stream=True).raw) for url in dict_images["urls"]]
class_images = dict_images["labels"]
prompt = "USER: <image>\nWhat is shown in the picture? ASSISTANT: The picture shows a"

Отобразим изображения

In [ ]:
plot_images(images)

Соберем данные для каждого из изображений: токены, вероятность, энтропия, косинусное расстояние между последним слоем и промежуточными слоями модели и внимание.

In [ ]:
data, data_attention = get_data_images(model, class_images, prompt, images, last_layer)

График зависимости вероятности предсказания правильного токена от номера слоя.

In [ ]:
plot_data(data, class_images, data_id=1, xlabel='n_layer', ylabel='prob')